# <font color = 'pickle'> **HW1  (20 Points):**</font> 

Required Submissions:
1.  Submit  colab/jupyter notebooks only.
2. **Name files as follows : FirstName_hw1**


In [94]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

In [95]:
torch.set_printoptions(precision=4, sci_mode=False)

 # <font color = 'pickle'>**Q1 : Create Tensor (1 Point)**
 Create a torch Tensor of shape (5, 3) which is filled with zeros. Modify the tensor to set element (0, 2) to 10 and element (2, 0)  to 100.

In [96]:
# code here
t1 = torch.zeros(size = (5,3))
t1

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [97]:
t1[0][2] = 10
t1[2][0] = 100

t1

tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])

# <font color = 'pickle'> **Q2: Reshape tensor (1 Point)**
You have following tensor as input:

```x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])```

Using only reshaping functions (like view, reshape, transpose, permute), you need to get at the following tensor as output:

```
tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])
```



In [98]:
x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [99]:
x_reshape = x.reshape(6,4)
torch.transpose(x_reshape,0,1)

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])

# <font color = 'pickle'>**Question 3. Speedtest for vectorization - 2 Points** </font>

Your goal is to measure the speed of linear algebra operations for different levels of vectorization.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $1024 \times 1024$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. (Hint: Use torch.mm)
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time. (hint use torch.mv inside a for loop)
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time. (Hint: use torch.dot inside nested for loop)

In [100]:
## Solution 1
torch.manual_seed(42) # dod not chnage this
A = torch.randn(size = (1024,1024))
B = torch.randn(size = (1024,1024))


In [101]:
## Solution 2
start=time.time()

C = torch.mm(A,B)
print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.03069019317626953 seconds


In [102]:
## Solution 3
C= torch.empty(1024,1024)
start = time.time()

for i in range(len(B[0])):
   C[:,i] = torch.mv(A,B[:,i])


print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.2459275722503662 seconds


In [103]:
## Solution 4
# code here
C= torch.empty(1024,1024)
start = time.time()
for i in range(len(A)):
  for j in range(len(B[0])):
    C[i,j] = torch.dot(A[i,:],B[:,j])

print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 20.970428466796875 seconds


# New Section

# <font color = 'pickle'>**Question 4 : Redo Question 3 by using GPU - 2 Point**

<font size = 4, color = 'dodgerblue'> **Using GPUs**

How to use GPUs in Google Colab<br>
In Google Colab -- Go to Runtime Tab at top -- select change runtime type -- for hardware accelartor choose GPU

In [104]:
# Check if GPU is availaible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [105]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((1024, 1024),device=device)
B= torch.randn((1024, 1024),device=device)

In [106]:
## Solution 2
start=time.time()

C = torch.mm(A,B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.0002810955047607422 seconds


In [107]:
## Solution 3
C= torch.empty(1024,1024, device = device)
start = time.time()

# code here
for i in range(len(B[0])):
  C[:,i] = torch.mv(A,B[:,i])

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.03442978858947754 seconds


In [108]:
## Solution 4
C= torch.empty(1024,1024, device = device)
start = time.time()

# code here
start = time.time()
for i in range(len(A)):
  for j in range(len(B[0])):
    C[i,j] = torch.dot(A[i],B[:,j])


print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 33.36388826370239 seconds


# <font color = 'pickle'>**Question 5. Memory efficient computation - 2 Points**</font>

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible. Hint: If you implement this correcly the memory location of C given by id(C) will be same in both the cells below.

In [109]:
A= torch.randn((1000, 1000),device=device)
B= torch.randn((1000,1000),device=device)
C= torch.randn((1000, 1000),device=device)
print(id(C))

140401609626928


In [110]:
# your code here
for i in range(len(A)):
  for j in range(len(B[0])):
    C[i,j] += torch.dot(A[i],B[:,j])
print(id(C))

140401609626928


#<font color = 'pickle'>**Question 6. Broadcast Operations - 2 Points**

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 3$ and $x = \{1,2,3,4,5\}$. Implement code that generates following A matrix

$$\begin{bmatrix} 1 & 1 & 1 \\ 2 & 4 & 8 \\ 3 & 9 & 27 \\ 4 & 16 & 64 \\ 5 & 25 & 125\end{bmatrix}$$

In [111]:
# your code here

x = torch.tensor([1,2,3,4,5]).reshape(-1,1)
y = torch.tensor([1,2,3])

print(x**y)




tensor([[  1,   1,   1],
        [  2,   4,   8],
        [  3,   9,  27],
        [  4,  16,  64],
        [  5,  25, 125]])


# <font color = 'pickle'>**Q7. Regression from scartch using Pytroch Tensors (5 Points)**</font>

Imagine that you're trying to figure out relationship between two variables x and y . You have some idea but you aren't quite sure yet whether the dependence is linear or quadratic. 

Your goal is to use least mean squares regression to identify the coefficients for the following three models using. The three models are:

1. Quadratic model where $\mathrm{y} = b + w_1 \cdot \mathrm{x} + w_2 \cdot \mathrm{x}^2$.
1. Linear model where $\mathrm{y} = b + w_1 \cdot \mathrm{x}$.
1. Linear model with no bias  where $\mathrm{y} = w_1 \cdot \mathrm{x}$.

- You will use batch gradient descent to estimate the model co-efficients. Batch gradient descent uses complete training data at each iteration. 
- We will implement only training loop (no splitting of data in to training/validation).
- The training loop will have only one for loop. We need to iterate over whole data in each epoch. We do not need to craete batches.
- You may have to try different values of number of epochs/ learning rate to get good results.
- You are not allowed to use Pytorch's nn.module or functions from Pytorch. You will write function for loss function (mean sqaured error), prediction and gradient update from scratch.

## <font color = 'pickle'> **Data**

In [118]:
x = torch.tensor([1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281])
y = torch.tensor([63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967])

In [119]:
x = x.unsqueeze(dim = 0)
y = y.unsqueeze(dim = 0)

In [120]:
#Initialize weights and bias
#weights - w, bias - b

torch.manual_seed(10)
wm = torch.randn(1, 2, requires_grad=False)
b = torch.randn(1, requires_grad=False)

#Since model2 and model3 have one weight, so using 1st weight ('w') for model2 and model3 and ('wm') for model1
w1 = wm.squeeze()[0]
w2 = wm.squeeze()[1]
w1 = w1[None,None]
w2 = w2[None,None]


In [121]:
#Model1 ( y = w1.x + w2.x.x + b )

def linear_regression_model1(X):
  return torch.mm(w1.T,X) + torch.mm(w2.T,X*X) + b

In [122]:
#Model2 ( y = w1.x + b )

def linear_regression_model2(X):
  return torch.mm(w1.T,X) + b

In [123]:
#Model3 ( y = w1.x )

def linear_regression_model3(X):
  return torch.mm(w1.T,X)

In [124]:
#Loss Function - Mean Squared Error

def mean_square_loss(ypred, y):
  error = ypred - y.view(ypred.shape)
  return torch.sum(error*error) / error.numel()

In [125]:
#Model1 Initial Predictions

prediction = linear_regression_model1(x)
loss = mean_square_loss(prediction,y)
loss

tensor(81521.9688)

In [126]:
#Model2 Initial Predictions

prediction = linear_regression_model2(x)
loss = mean_square_loss(prediction,y)
loss

tensor(59741.3359)

In [127]:
#Model3 Initial Predictions

prediction = linear_regression_model3(x)
loss = mean_square_loss(prediction,y)
loss

tensor(59606.7695)

# Model 1 - Training and Checking results

In [128]:
#Training the Model1

epochs = 5000
lr = 0.0000265

#Initialize weights and bias
#weights - w, bias - b

torch.manual_seed(10)
wm = torch.randn(1, 2, requires_grad=False)
b = torch.randn(1, requires_grad=False)

#Since model2 and model3 have one weight, so using 1st weight ('w') for model2 and model3 and ('wm') for model1
w1 = wm.squeeze()[0]
w2 = wm.squeeze()[1]
w1 = w1[None,None]
w2 = w2[None,None]


#Train on weights and biases
weight_2_1 = w1
weight_2_2 = w2
bias_1 = b

for i in range(epochs):
  prediction1 = linear_regression_model1(x)
  loss1 = mean_square_loss(prediction1, y)
  grad_w_2_1 = 2 * torch.sum(x*(prediction1 - y.view(prediction1.shape))) / len(y)
  grad_w_2_2 = 2 * torch.sum((x*x)*(prediction1 - y.view(prediction1.shape))) / len(y)
  grad_b_1 =  2 * torch.sum(prediction1 - y.view(prediction1.shape)) / len(y)

  weight_2_1 -= grad_w_2_1 * lr
  weight_2_2 -= grad_w_2_2 * lr
  bias_1 -= grad_b_1 * lr
  print("weight1 = {}, weight2 = {}, bias = {}, loss = {}".format (weight_2_1, weight_2_2, bias_1, loss1))

weight1 = tensor([[1.1295]]), weight2 = tensor([[11.3344]]), bias = tensor([-0.0310]), loss = 81521.96875
weight1 = tensor([[-0.2270]]), weight2 = tensor([[0.2631]]), bias = tensor([-0.1983]), loss = 66029.984375
weight1 = tensor([[1.1777]]), weight2 = tensor([[10.1403]]), bias = tensor([0.0265]), loss = 53628.61328125
weight1 = tensor([[0.1119]]), weight2 = tensor([[1.2781]]), bias = tensor([-0.0997]), loss = 43700.41796875
weight1 = tensor([[1.2555]]), weight2 = tensor([[9.1793]]), bias = tensor([0.0879]), loss = 35751.375
weight1 = tensor([[0.4222]]), weight2 = tensor([[2.0849]]), bias = tensor([-0.0054]), loss = 29386.068359375
weight1 = tensor([[1.3568]]), weight2 = tensor([[8.4049]]), bias = tensor([0.1523]), loss = 24288.111328125
weight1 = tensor([[0.7094]]), weight2 = tensor([[2.7251]]), bias = tensor([0.0853]), loss = 20204.34765625
weight1 = tensor([[1.4767]]), weight2 = tensor([[7.7797]]), bias = tensor([0.2192]), loss = 16932.17578125
weight1 = tensor([[0.9779]]), weight2 

In [129]:
#Prediction after training the model

prediction1 = torch.mm(weight_2_1.T,x) + torch.mm(weight_2_2.T,x*x) + bias_1
prediction1

tensor([[ 70.6554,  84.2814,  94.6875, 109.5167, 123.5584, 161.3941, 164.8926,
         176.9771, 177.4734, 179.0128, 206.0431, 270.2940, 272.2909, 287.6403,
         292.8819, 294.3730, 304.7964, 325.4143, 385.3093, 412.4066]])

In [130]:
#Coefficients 

print("w1_coefficent : ", weight_2_1)
print("w2_coefficient:", weight_2_2)
print("bias : ", bias_1)

w1_coefficent :  tensor([[37.4587]])
w2_coefficient: tensor([[0.3805]])
bias :  tensor([11.9882])


In [131]:
#Actual Target Values - We see that the predicted values from above are very close to y

print(y)

tensor([[ 63.8022,  80.0360,  91.4903, 108.2878, 122.7820, 161.3631, 166.5082,
         176.1677, 180.2939, 179.0976, 206.2103, 272.7186, 272.2403, 289.5475,
         293.8488, 295.2281, 306.6227, 327.9324, 383.1630, 408.6597]])


# Model 2 - Training and Checking Results

In [132]:
#Training the Model2

epochs = 300
lr = 0.001

#Initialize weights and bias
#weights - w, bias - b

torch.manual_seed(10)
wm = torch.randn(1, 2, requires_grad=False)
b = torch.randn(1, requires_grad=False)

#Since model2 and model3 have one weight, so using 1st weight ('w') for model2 and model3 and ('wm') for model1
w1 = wm.squeeze()[0]
w2 = wm.squeeze()[1]
w1 = w1[None,None]
w2 = w2[None,None]

#Train on weights and biases
weight_2 = w1
bias_2 = b

for i in range(epochs):
  prediction2 = linear_regression_model2(x)
  loss2 = mean_square_loss(prediction2, y)
  grad_w_2 = 2 * torch.sum(x*(prediction2 - y.view(prediction2.shape))) / len(y)
  grad_b_2 =  2 * torch.sum(prediction2 - y.view(prediction2.shape)) / len(y)

  weight_2 -= grad_w_2 * lr
  bias_2 -= grad_b_2 * lr
  print("weight = {}, bias = {}, loss = {}".format (weight_2,bias_2, loss2))

weight = tensor([[55.3531]]), bias = tensor([8.6072]), loss = 59741.3359375
weight = tensor([[36.1374]]), bias = tensor([5.4944]), loss = 7057.70458984375
weight = tensor([[42.7475]]), bias = tensor([6.5124]), loss = 839.0628662109375
weight = tensor([[40.4847]]), bias = tensor([6.1115]), loss = 104.9727554321289
weight = tensor([[41.2702]]), bias = tensor([6.1984]), loss = 18.258548736572266
weight = tensor([[41.0084]]), bias = tensor([6.1181]), loss = 7.958963871002197
weight = tensor([[41.1064]]), bias = tensor([6.0955]), loss = 6.680010795593262
weight = tensor([[41.0807]]), bias = tensor([6.0535]), loss = 6.466641902923584
weight = tensor([[41.0974]]), bias = tensor([6.0185]), loss = 6.3798394203186035
weight = tensor([[41.0995]]), bias = tensor([5.9814]), loss = 6.308797359466553
weight = tensor([[41.1066]]), bias = tensor([5.9453]), loss = 6.2404351234436035
weight = tensor([[41.1119]]), bias = tensor([5.9092]), loss = 6.173191070556641
weight = tensor([[41.1178]]), bias = tenso

In [133]:
#Prediction after training the model

prediction = torch.mm(weight_2.T,x) + bias_2
prediction

tensor([[ 65.8460,  80.5548,  91.7199, 107.5316, 122.3978, 161.9599, 165.5825,
         178.0504, 178.5610, 180.1439, 207.7579, 272.0837, 274.0545, 289.1480,
         294.2801, 295.7380, 305.9042, 325.8856, 383.0044, 408.4111]])

In [134]:
#Coefficients 

print("w_coefficent : ", weight_2)
print("bias : ", bias_2)

w_coefficent :  tensor([[41.8464]])
bias :  tensor([1.3177])


In [135]:
#Actual Target Values - We see that the predicted values from above are very close to y

print(y)

tensor([[ 63.8022,  80.0360,  91.4903, 108.2878, 122.7820, 161.3631, 166.5082,
         176.1677, 180.2939, 179.0976, 206.2103, 272.7186, 272.2403, 289.5475,
         293.8488, 295.2281, 306.6227, 327.9324, 383.1630, 408.6597]])


# Model 3 - Training and checking results



In [136]:
#Training the Model3

epochs = 10
lr = 0.001


#Initialize weights and bias
#weights - w, bias - b

torch.manual_seed(10)
wm = torch.randn(1, 2, requires_grad=False)
b = torch.randn(1, requires_grad=False)

#Since model2 and model3 have one weight, so using 1st weight ('w') for model2 and model3 and ('wm') for model1
w1 = wm.squeeze()[0]
w2 = wm.squeeze()[1]
w1 = w1[None,None]
w2 = w2[None,None]
print(w1,w2,b)

#Train on weights and biases
weight_3 = w1

for i in range(epochs):
  prediction3 = linear_regression_model3(x)
  loss3 = mean_square_loss(prediction3, y)
  grad_w_3 = 2 * torch.sum(x*(prediction3 - y.view(prediction3.shape))) / len(y)

  weight_3 -= grad_w_3 * lr
  print("weight = {}, loss = {}".format(weight_3, loss3))

tensor([[-0.6014]]) tensor([[-1.0122]]) tensor([-0.3023])
weight = tensor([[55.2900]]), loss = 59606.76953125
weight = tensor([[37.9515]]), loss = 5737.39306640625
weight = tensor([[43.3302]]), loss = 553.2445068359375
weight = tensor([[41.6616]]), loss = 54.34516143798828
weight = tensor([[42.1793]]), loss = 6.333302974700928
weight = tensor([[42.0187]]), loss = 1.7128393650054932
weight = tensor([[42.0685]]), loss = 1.2681820392608643
weight = tensor([[42.0530]]), loss = 1.225394606590271
weight = tensor([[42.0578]]), loss = 1.221276044845581
weight = tensor([[42.0564]]), loss = 1.220876693725586


In [137]:
#Prediction after training the model

prediction3 = torch.mm(weight_3.T,x)
prediction3

tensor([[ 64.8521,  79.6347,  90.8559, 106.7469, 121.6877, 161.4482, 165.0891,
         177.6195, 178.1327, 179.7235, 207.4761, 272.1246, 274.1053, 289.2745,
         294.4323, 295.8976, 306.1148, 326.1965, 383.6018, 409.1360]])

In [138]:
#Coefficients 

print("w_coefficent : ", weight_3)


w_coefficent :  tensor([[42.0564]])


In [139]:
#Actual Target Values - We see that the predicted values from above are very close to y

print(y)

tensor([[ 63.8022,  80.0360,  91.4903, 108.2878, 122.7820, 161.3631, 166.5082,
         176.1677, 180.2939, 179.0976, 206.2103, 272.7186, 272.2403, 289.5475,
         293.8488, 295.2281, 306.6227, 327.9324, 383.1630, 408.6597]])


# <font color = 'pickle'> **Q8 Image Classification using - Pixel Similarity - 5 Points**

## <font color = 'pickle'> **Import libraries**

In [140]:
# import libraries
import torchvision
import torchvision.transforms as transforms
from pathlib import Path
import cv2 as cv 
from google.colab.patches import cv2_imshow # for image display
import pandas as pd

## <font color = 'pickle'> **Mount Google Deive**

In [141]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## <font color = 'pickle'>**Path to Dowanload Data**

<font color = 'dodgerblue'>**Make sure you change the Path to where you want to save data.**</font>
In the code below - **data/datasets** is the folder name in my google drive. You can change this to appropriate folder for your drive. For example you may want to save data to BUAN6341/HW1/Data. In this case the below code should be modified to :

 ```data_folder = Path('/content/drive/MyDrive/BUAN6341/HW2/Data')```

In [142]:
data_folder = Path('/content/drive/MyDrive/Deep_Learning_UTD/Dataset')

## <font color = 'pickle'>**Download MNIST training Data**
The MNIST is a dataset of  handwritten digits, available from this page http://yann.lecun.com/exdb/mnist/. It has a training set of 60,000 examples, and a test set of 10,000 examples. In this task we will use images for digits 3 and 7.

In [143]:
trainset = torchvision.datasets.MNIST(root = data_folder,
                                      train = True,
                                      download = True,
                                      )

## <font color = 'pickle'>**Subset of images**

trainset.data has images and trainset.targets has the labels. Below we will create a Boolean mask for digits 3 and 7. We will use this mask to extract subset of images.

In [144]:
# get the mask 
idx3 = trainset.targets==3
idx7 = trainset.targets==7

In [145]:
idx3 # This take the value of True wherever the label was 3 and False otherwise

tensor([False, False, False,  ..., False, False, False])

In [146]:
threes = trainset.data[idx3] # use Boolean mask to extract images for digit three
sevens = trainset.data[idx7] # use Boolean mask to extract images for digit seven

In [147]:
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [148]:
type(threes[0])

torch.Tensor

In [149]:
# get a sample image
sample_img_3 = threes[0]

In [150]:
sample_img_3.shape

torch.Size([28, 28])

### <font color = 'pickle'>**Task1**
Reshape the image so that it has following  shape: 28, 28, 1. Then convert the tensor to numpy array. 

In [151]:
sample_img_3_numpy =  np.array(sample_img_3.unsqueeze(dim = 2))

In [152]:
# let us look at the image
# the above steps were needed because the function cv2_imshow needs a three dimenional numpy array of the 
# shape H x W  X C (height, width, number of channels). The number of channels for black and white images is 1.
cv2_imshow(sample_img_3_numpy)

### <font color = 'pickle'>**Task2**

Repeat the above steps to show the first image for digit 7.
 - Get the first image.
 - Reshape the image to 28, 28, 1
 - Convert the tensor to numpy array


In [153]:
sample_img_7 = sevens[0]
sample_img_7.shape

torch.Size([28, 28])

In [154]:
# code here
sample_img_7_numpy = np.array(sample_img_7.unsqueeze(dim = 2))

In [155]:
cv2_imshow(sample_img_7_numpy)

## <font color = 'pickle'>**Visualizing images as pixel values**

<font color = 'dodgerblue'> 
- In a computer, images are stored in the form of matrices. The numbers in the matrix are called "pixel values."
- These pixel values show how bright each pixel is.
- 0 means "black," and 255 means "white."
- The matrix of numbers is called the channel, and there is only one channel in a grayscale image.
- For color inages , we have three channels(RGB - Red, Green Blue). That is we need three matrices to store color images.
</font>

In [156]:
df = pd.DataFrame(sample_img_3[5:24, 7:22].numpy())
df.style.set_properties(**{'font-size':'8pt'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


<font color = 'dodgerblue'> - In the matrix below we have flipped color just for illustartion (We are using 0 for white instead of black).

In [157]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


### <font color = 'pickle'>**Task3**

<font color = 'pickle'>**Convert threes and sevens generated earlier to float and and take the mean along dimension 0.**

Hint(You can use tensor.float() and then take the mean along dimesnion 0.

Here we are generating an average image by taking the average of each pixel across images.

In [158]:
mean_threes =  torch.mean(threes.float(), dim = 0)
mean_sevens =  torch.mean(sevens.float(), dim = 0)

mean_threes

tensor([[    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000],
        [    0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
             0.0000,     0.0000,     0.0000,     0.0000,     0.0186,     0.0501,
             0.0597,     0.0491,     0.0703,     0.0667,     0.0750,     0

In [159]:
mean_threes.shape
mean_sevens.shape

torch.Size([28, 28])

In [435]:
cv2_imshow(mean_threes.unsqueeze(dim=2).numpy())
print()
cv2_imshow(mean_sevens.unsqueeze(dim=2).numpy())

## <font color = 'pickle'>**Prediction for validation images**

We will calculate the distance of each validation image from the mean_threes and mean_sevens. If the validation image is closest to mean_threes then we will predict 1 else we will predict 0.

## <font color = 'pickle'>**Get Valid Dataset**

In [160]:
validset = torchvision.datasets.MNIST(root = data_folder,
                                      train = False,
                                      download = True,
                                      )

In [161]:
# get the mask 
idx3 = validset.targets==3
idx7 = validset.targets==7

In [162]:
validset.data.shape

torch.Size([10000, 28, 28])

In [163]:
# get images and labels using the mask
valid_data_3_7 = validset.data[idx3+idx7]
valid_targets_3_7 = validset.targets[idx3 + idx7]

In [164]:
# check the shape of the inputs (images)
# we have 2038 images with size 28 x 28.
valid_data_3_7.shape

torch.Size([2038, 28, 28])

In [165]:
# lables for valid dataset
valid_targets_3_7

tensor([7, 7, 3,  ..., 3, 7, 3])

In [166]:
# change the lable to 1 where the label was 3
valid_targets_3_7[valid_targets_3_7==3] = 1

In [167]:
# change the lable to 0 where the label was 7
valid_targets_3_7[valid_targets_3_7==7] = 0

In [168]:
valid_targets_3_7.unique()

tensor([0, 1])

In [169]:
valid_targets_3_7

tensor([0, 0, 1,  ..., 1, 0, 1])

In [170]:
valid_data_3_7.shape

torch.Size([2038, 28, 28])

## <font color = 'pickle'>**Distance between images**

### <font color = 'pickle'>**Task4** 

<font color = 'pickle'>**Write a function to calculate the distance between two images a and b.** 
 - Calculate the difference between corresponding pixels of two images.
 - Calculate square of differences.
 - Take the mean (take the mean across last two dimensions (-1, -2) of the square of the differences and then take the square root.

In [171]:
def dist_images(a, b):
  diff_sq = (a - b)**2
  mean_diff_sq = torch.mean(diff_sq.float(), dim = (-1,-2))
  return (mean_diff_sq)**(0.5)

In [172]:
# calculate the distance of the each image in validation set from mean_threes
# bacause of the  broadcasting we are able to use the above function which was written to 
# calculate the distnace between two images
valid_3_dist = dist_images(valid_data_3_7, mean_threes)
valid_3_dist

tensor([70.2972, 71.7095, 79.9791,  ..., 59.1907, 68.7125, 58.1954])

In [173]:
# calculate the distance of the each image in validation set from mean_sevens
valid_7_dist = dist_images(valid_data_3_7, mean_sevens)

In [174]:
# if you have done everything right, you will observe following shapes.
valid_3_dist.shape, valid_7_dist.shape

(torch.Size([2038]), torch.Size([2038]))

## <font color = 'pickle'>**Prediction Using Distance**

### <font color = 'pickle'>**Task5**



In [175]:
# step 1 : generate the tensor filled with zeros taht has the same shape as valid_target_3_7
prediction =  torch.zeros(size = (2038,))

In [176]:
# step 2: update the tensor prediction. Update the value to 1, when valid_3_dist < valid_7_dist.
# Hint (Generate a Boolean mask using the condition valid_3_dist < valid_7_dist and use the mask to update value to 1 )

for i in range(len(valid_3_dist)):
  prediction[i] = 1 if valid_3_dist[i] < valid_7_dist[i] else 0 

In [177]:
prediction

tensor([0., 0., 1.,  ..., 1., 0., 1.])

## <font color = 'pickle'>**Accuracy**

In [178]:
correct = prediction == valid_targets_3_7

In [179]:
correct

tensor([True, True, True,  ..., True, True, True])

In [180]:
100 * correct.float().mean().item()

96.6143250465393